## Cousera-Capstone-Project Wk 3
***

This is the capstone project for IBM's Data Science Professional Certificate. In this project, we will try and use location data to solve an interesting business problem.

# Introduction/Business Problem:
***

A business owner is looking to open up a Cafe in Singapore and is deciding where to open up the cafe.

# Data Sources:
***

In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import urllib
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

Source 2 Dataframe

In [3]:
path= 'https://raw.githubusercontent.com/erictay49/Final_Capstone_IBM/master/Substate_Cleaned.csv'
df = pd.read_csv(path)
df['% Total Pop']=df['Total Pop']/df['Total Pop'].sum()
df['% Working Pop']= df['Working']/df['Total Pop']
df['% Elderly']= df['Elderly']/df['Total Pop']
df['% Kids']=df['Kids']/df['Total Pop']
df['% High']=df['Landed Properties']/df['Total Pop']
df['% Mid']=(df['Condos']+df['5-Rm-HDB & EC']+df['4-Rm-HDB'])/df['Total Pop']
df['% Low']=(df['1/2-Rm-HDB']+df['3-Rm-HDB'])/df['Total Pop']
df=df.fillna(0)
df

,Region,County,0 - 4,5-9,10-14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,45 - 49,50 - 54,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 - 89,90 & Over,1/2-Rm-HDB,3-Rm-HDB,4-Rm-HDB,5-Rm-HDB & EC,Condos,Landed Properties,Others,Total Pop,Latitude,Longitude,Kids,Youth,Working,Elderly,% Total Pop,% Working Pop,% Elderly,% Kids,% High,% Mid,% Low
0,Ang Mo Kio,Ang Mo Kio Town Centre,170,240,310,310,280,280,280,320,420,470,350,350,280,260,210,140,80,40,20,0,420,630,1830,1900,0,30,4810,1.3731,103.8467,720,590,2750,750,0.001194,0.571726,0.155925,0.149688,0.000000,0.906445,0.087318
1,Ang Mo Kio,Cheng San,1060,1000,1070,1210,1370,1800,2000,2210,2040,2230,2080,2160,2200,2140,1560,1020,590,310,160,950,13390,8800,4930,0,0,140,28210,1.3729,103.8540,3130,2580,16720,5780,0.007005,0.592698,0.204892,0.110954,0.000000,0.486707,0.508330
2,Ang Mo Kio,Chong Boon,860,880,1040,1140,1350,1610,1920,1820,1890,1830,1980,2140,2160,2070,1670,1180,730,390,180,1580,12370,8620,3770,0,0,500,26840,1.3628,103.8560,2780,2490,15350,6220,0.006665,0.571908,0.231744,0.103577,0.000000,0.461624,0.519747
3,Ang Mo Kio,Kebun Bahru,730,840,1030,1080,1160,1430,1390,1530,1670,1840,1800,1780,1740,1710,1300,890,590,310,150,1690,10430,5720,2170,1580,1140,240,22970,1.3730,103.8392,2600,2240,13180,4950,0.005704,0.573792,0.215498,0.113191,0.049630,0.412277,0.527645
4,Ang Mo Kio,Sembawang Hills,210,290,410,470,540,520,320,300,400,550,540,560,480,420,310,240,150,100,50,0,0,0,0,250,6510,100,6860,1.3743,103.8298,910,1010,3670,1270,0.001703,0.534985,0.185131,0.132653,0.948980,0.036443,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Yishun,Springleaf,130,190,240,300,330,400,240,180,260,330,300,380,320,280,230,100,70,30,20,0,0,0,0,130,4180,20,4330,1.4030,103.8227,560,630,2410,730,0.001075,0.556582,0.168591,0.129330,0.965358,0.030023,0.000000
319,Yishun,Yishun Central,60,90,100,90,140,180,150,160,160,140,150,190,150,120,70,40,20,10,10,0,0,720,600,700,0,10,2030,1.4267,103.8379,250,230,1280,270,0.000504,0.630542,0.133005,0.123153,0.000000,0.995074,0.000000
320,Yishun,Yishun East,4610,4280,3120,3020,3270,4110,5820,6290,4660,3950,3880,4020,3270,2410,1390,750,450,220,110,3470,7270,32030,13250,3580,0,30,59630,1.4279,103.8476,12010,6290,36000,5330,0.014807,0.603723,0.089385,0.201409,0.000000,0.819386,0.180111
321,Yishun,Yishun South,1850,1650,1960,2150,2570,3920,3470,2880,2790,3010,3100,3720,3190,2410,1520,900,530,270,170,950,4530,20120,13340,2890,0,230,42060,1.4186,103.8386,5460,4720,26080,5800,0.010444,0.620067,0.137898,0.129815,0.000000,0.864242,0.130290


## Scoring Each District
---

The above 3 features are all likely to contribute to more spending on coffee/cafe visits. As we have no clear way of determining the relationship with coffee visits, let's evenly weigh each feature in scoring. Scores are determined by dividing the values by their mean values, and multiplied by 0.333 (even weightage).

In [4]:
pop_med = df['Total Pop'].max()
income_med = df['% Mid'].max()
working_med = df['% Working Pop'].max()

df_score = pd.DataFrame(columns=['Region','County',"pop_score", "income_score", "work_score", "total_score"])
df_score['Region'] = df['Region']
df_score['County'] = df['County']

pop_score = []
for x in df['Total Pop']:
  if x / pop_med >0:
    pop_score.append((x / pop_med)*.333)
  else:
    pop_score.append(0)
df_score['pop_score'] = pop_score

income_score = []
for x in df['% Mid']:
  if x / income_med >0:
    income_score.append((x / income_med)*.333)
  else:
    income_score.append(0)
df_score['income_score'] = income_score

work_score = []
for x in df['% Working Pop']:
  if x / working_med >0:
    work_score.append((x / working_med)*.333)
  else:
    work_score.append(0)
df_score['work_score'] = work_score

df_score['total_score'] = round(df_score.iloc[:,-5:].sum(axis=1),2)
df_score= df_score.sort_values('total_score',ascending=False)
df_score.head()

,Region,County,pop_score,income_score,work_score,total_score
274,Tampines,Tampines East,0.333000,0.262098,0.239874,0.83
310,Woodlands,Woodlands East,0.246980,0.315760,0.240547,0.80
128,Jurong West,Jurong West Central,0.162138,0.329447,0.246359,0.74
276,Tampines,Tampines West,0.199795,0.282104,0.239690,0.72
249,Sengkang,Sengkang Town Centre,0.154516,0.319708,0.242626,0.72


## Final Dataframe for Analysis: 

In [5]:
df_final_t=pd.merge(df,df_score)
df_final_t=df_final_t.sort_values('total_score', ascending=False)
df_final_t=df_final_t.rename(columns={'County': 'neighborhood'})
df_final=df_final_t[['Region','neighborhood','Latitude','Longitude','Total Pop','pop_score','income_score','work_score','total_score']]
df_final.head()

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score
274,Tampines,Tampines East,1.3535,103.9527,132820,0.333000,0.262098,0.239874,0.83
310,Woodlands,Woodlands East,1.4413,103.8033,98510,0.246980,0.315760,0.240547,0.80
128,Jurong West,Jurong West Central,1.3409,103.7025,64670,0.162138,0.329447,0.246359,0.74
276,Tampines,Tampines West,1.3483,103.9337,79690,0.199795,0.282104,0.239690,0.72
249,Sengkang,Sengkang Town Centre,1.3914,103.8972,61630,0.154516,0.319708,0.242626,0.72


# MAPS! Visualizing the data
***

In [6]:
! pip install folium
import folium

In [7]:
m=folium.Map([1.4425,103.7958],zoom_start=11)
for lat,lng,region,neighborhood in zip(df_final['Latitude'],df_final['Longitude'],df_final['Region'],df_final['neighborhood']):
    label = '{}, {}'.format(region,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], radius=5, popup=label, color='red', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(m)
display(m)

In [ ]:
CLIENT_ID = 'A5S2CJNU43XNBJEADGVEDLOR024ZP5BC5KZY2E1F0WT0DZEI' # your Foursquare ID
CLIENT_SECRET = 'GIPWZSDNB1GYTVSRWTFV2E2JZBHBDYCORNL3MVRVDUOWQADI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=30

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=150):
    
    venues_list=[]
    categoryID='4bf58dd8d48988d16d941735' #Bubble Tea
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryID)
        #print(requests.get(url).json())
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print('Your quota may have been exceeded')
            return
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
SG_Venues= getNearbyVenues(names=df_final['neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude'])
print(SG_Venues.shape)
SG_Venues.head()

In [ ]:
SG_Venues["Venue Category"].value_counts()

Café                         371
Coffee Shop                   15
Bakery                         6
Asian Restaurant               4
Food Court                     3
Dessert Shop                   3
Pub                            2
Bar                            2
Restaurant                     2
Noodle House                   1
Mexican Restaurant             1
BBQ Joint                      1
Nail Salon                     1
Bubble Tea Shop                1
Lounge                         1
Italian Restaurant             1
Cafeteria                      1
Juice Bar                      1
French Restaurant              1
Diner                          1
College Cafeteria              1
Boutique                       1
Middle Eastern Restaurant      1
Gay Bar                        1
Hotel                          1
Name: Venue Category, dtype: int64

In [ ]:
SG_onehot = pd.get_dummies(SG_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SG_onehot['neighborhood'] = SG_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [SG_onehot.columns[-1]] + list(SG_onehot.columns[:-1])
SG_onehot = SG_onehot[fixed_columns]
SG_onehot

,neighborhood,Asian Restaurant,BBQ Joint,Bakery,Bar,Boutique,Bubble Tea Shop,Cafeteria,Café,Coffee Shop,College Cafeteria,Dessert Shop,Diner,Food Court,French Restaurant,Gay Bar,Hotel,Italian Restaurant,Juice Bar,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Nail Salon,Noodle House,Pub,Restaurant
0,Fernvale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Waterway East,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Waterway East,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Waterway East,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hougang West,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Bayfront Subzone,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
420,Bayfront Subzone,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
421,Bayfront Subzone,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
422,Bayfront Subzone,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
SG_grouped = pd.DataFrame(SG_onehot.groupby('neighborhood')["Café","Coffee Shop"].sum().sum(axis=1))
SG_grouped.columns = ['No. of Coffee Shops']
SG_grouped=SG_grouped.sort_values('No. of Coffee Shops',ascending=False)
SG_grouped


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,No. of Coffee Shops
neighborhood,
Somerset,28
Boulevard,26
Raffles Place,24
Tanjong Pagar,24
City Hall,19
...,...
Sembawang Straits,0
Macpherson,0
Toa Payoh Central,0


In [ ]:
df_Coffee_match=pd.merge(df_final,SG_grouped,on='neighborhood',how='left')
df_Coffee_match=df_Coffee_match.fillna(0).sort_values('total_score',ascending=False)
df_Coffee_match

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score,No. of Coffee Shops
0,Tampines,Tampines East,1.3535,103.9527,132820,3.547357,0.548001,0.467472,4.56,0.0
1,Woodlands,Woodlands East,1.4413,103.8033,98510,2.631005,0.660199,0.468783,3.76,0.0
2,Tampines,Tampines West,1.3483,103.9337,79690,2.128361,0.589829,0.467112,3.19,0.0
3,Bedok,Bedok North,1.3297,103.9371,82520,2.203944,0.351413,0.452225,3.01,0.0
4,Jurong West,Yunnan,1.3450,103.6949,68190,1.821219,0.619157,0.469259,2.91,0.0
...,...,...,...,...,...,...,...,...,...,...
261,Jurong West,Kian Teck,1.3337,103.6996,0,0.000000,0.000000,0.000000,0.00,0.0
260,Toa Payoh,Bidadari,1.3411,103.8756,0,0.000000,0.000000,0.000000,0.00,0.0
259,Ang Mo Kio,Yio Chu Kang North,1.3926,103.8472,0,0.000000,0.000000,0.000000,0.00,0.0
258,Boon Lay,Liu Fang,1.3188,103.7083,0,0.000000,0.000000,0.000000,0.00,0.0


In [ ]:
kclusters = 5

df_Venues= df_Coffee_match[['neighborhood','total_score', 'No. of Coffee Shops']]

df_Coffee_clustering = df_Venues.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_Coffee_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
df_Venues.insert(0, 'ClustersL', kmeans.labels_)

df_Coffee_match_final= pd.merge(df_Coffee_match,df_Venues)
df_Coffee_match_final.sort_values('No. of Coffee Shops')

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score,No. of Coffee Shops,ClustersL
0,Tampines,Tampines East,1.3535,103.9527,132820,3.547357,0.548001,0.467472,4.56,0.0,0
195,Serangoon,Seletar Hills,1.3838,103.8676,14240,0.380322,0.095343,0.433671,0.91,0.0,0
196,Rochor,Little India,1.3068,103.8542,3390,0.090540,0.375849,0.443358,0.91,0.0,0
197,Bukit Merah,Kampong Tiong Bahru,1.2784,103.8272,9220,0.246248,0.224278,0.430761,0.90,0.0,0
198,Ang Mo Kio,Tagore,1.3860,103.8293,8070,0.215534,0.218278,0.447756,0.88,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
312,Downtown Core,City Hall,1.2928,103.8527,0,0.000000,0.000000,0.000000,0.00,19.0,2
127,Downtown Core,Tanjong Pagar,1.2762,103.8462,630,0.016826,0.607833,0.580971,1.21,24.0,1
307,Downtown Core,Raffles Place,1.2836,103.8511,0,0.000000,0.000000,0.000000,0.00,24.0,1
186,Orchard,Boulevard,1.3051,103.8311,460,0.012286,0.544887,0.457091,1.01,26.0,1


In [ ]:
map_clusters = folium.Map([1.4425,103.7958],zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, pop in zip(df_Coffee_match_final['Latitude'], df_Coffee_match_final['Longitude'], df_Coffee_match_final['neighborhood'], df_Coffee_match_final['ClustersL'],df_Coffee_match_final['Total Pop']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster+1)+ ","+str(pop), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Cluster Description
***

## Cluster 1

In [ ]:
df_Coffee_match_final[df_Coffee_match_final['ClustersL']==0].sort_values('total_score', ascending=False)

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score,No. of Coffee Shops,ClustersL
0,Tampines,Tampines East,1.3535,103.9527,132820,3.547357,0.548001,0.467472,4.56,0.0,0
1,Woodlands,Woodlands East,1.4413,103.8033,98510,2.631005,0.660199,0.468783,3.76,0.0,0
2,Tampines,Tampines West,1.3483,103.9337,79690,2.128361,0.589829,0.467112,3.19,0.0,0
3,Bedok,Bedok North,1.3297,103.9371,82520,2.203944,0.351413,0.452225,3.01,0.0,0
4,Jurong West,Yunnan,1.3450,103.6949,68190,1.821219,0.619157,0.469259,2.91,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
196,Rochor,Little India,1.3068,103.8542,3390,0.090540,0.375849,0.443358,0.91,0.0,0
197,Bukit Merah,Kampong Tiong Bahru,1.2784,103.8272,9220,0.246248,0.224278,0.430761,0.90,0.0,0
198,Ang Mo Kio,Tagore,1.3860,103.8293,8070,0.215534,0.218278,0.447756,0.88,0.0,0
199,Singapore River,Clarke Quay,1.2898,103.8449,130,0.003472,0.267786,0.599037,0.87,3.0,0


## Cluster 2

In [ ]:
df_Coffee_match_final[df_Coffee_match_final['ClustersL']==1].sort_values('total_score', ascending=False)

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score,No. of Coffee Shops,ClustersL
127,Downtown Core,Tanjong Pagar,1.2762,103.8462,630,0.016826,0.607833,0.580971,1.21,24.0,1
182,Orchard,Somerset,1.3015,103.8391,150,0.004006,0.556996,0.467249,1.03,28.0,1
186,Orchard,Boulevard,1.3051,103.8311,460,0.012286,0.544887,0.457091,1.01,26.0,1
307,Downtown Core,Raffles Place,1.2836,103.8511,0,0.000000,0.000000,0.000000,0.00,24.0,1


## Cluster 3

In [ ]:
df_Coffee_match_final[df_Coffee_match_final['ClustersL']==2].sort_values('total_score', ascending=False)

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score,No. of Coffee Shops,ClustersL
101,Downtown Core,Cecil,1.2785,103.8486,240,0.006410,0.638224,0.648957,1.29,14.0,2
120,Museum,Dhoby Ghaut,1.2991,103.8453,280,0.007478,0.646513,0.584061,1.24,14.0,2
134,Outram,Chinatown,1.2797,103.8422,10650,0.284440,0.448473,0.457743,1.19,12.0,2
157,Downtown Core,Bugis,1.2992,103.8568,890,0.023770,0.641484,0.463749,1.13,11.0,2
218,Rochor,Kampong Glam,1.3024,103.8596,130,0.003472,0.107115,0.479230,0.59,13.0,2
220,Outram,China Square,1.2843,103.8470,1430,0.038192,0.058426,0.441109,0.54,18.0,2
265,Downtown Core,Anson,1.2741,103.8433,0,0.000000,0.000000,0.000000,0.00,13.0,2
310,Downtown Core,Marina Centre,1.2931,103.8602,0,0.000000,0.000000,0.000000,0.00,15.0,2
312,Downtown Core,City Hall,1.2928,103.8527,0,0.000000,0.000000,0.000000,0.00,19.0,2


## Cluster 4

In [ ]:
df_Coffee_match_final[df_Coffee_match_final['ClustersL']==3].sort_values('Total Pop', ascending=False)

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score,No. of Coffee Shops,ClustersL
61,Bishan,Upper Thomson,1.3554,103.8300,29430,0.786017,0.416848,0.439518,1.64,8.0,3
67,Marine Parade,Marine Parade,1.3024,103.9062,26880,0.717911,0.442406,0.422112,1.58,7.0,3
208,Rochor,Victoria,1.2999,103.8532,1900,0.050745,0.271169,0.450854,0.77,9.0,3
211,Rochor,Bencoolen,1.3017,103.8514,1300,0.034720,0.208873,0.467249,0.71,5.0,3
135,Jurong East,Jurong Gateway,1.3326,103.7431,490,0.013087,0.653618,0.508570,1.18,8.0,3
181,Rochor,Selegie,1.3024,103.8496,250,0.006677,0.556996,0.467249,1.03,8.0,3
164,Singapore River,Boat Quay,1.2877,103.8482,100,0.002671,0.626620,0.467249,1.10,7.0,3
234,Bukit Merah,Singapore General Hospital,1.2797,103.8345,0,0.000000,0.000000,0.000000,0.00,4.0,3
264,Downtown Core,Bayfront Subzone,1.2829,103.8582,0,0.000000,0.000000,0.000000,0.00,6.0,3
267,Museum,Bras Basah,1.2958,103.8500,0,0.000000,0.000000,0.000000,0.00,4.0,3


In [ ]:
df_Coffee_match_final[df_Coffee_match_final['ClustersL']==4]

,Region,neighborhood,Latitude,Longitude,Total Pop,pop_score,income_score,work_score,total_score,No. of Coffee Shops,ClustersL
200,Ang Mo Kio,Yio Chu Kang East,1.3840,103.8537,4230,0.112975,0.301212,0.441843,0.86,0.0,4
201,Queenstown,Pasir Panjang 2,1.2775,103.7955,3060,0.081726,0.320819,0.447907,0.85,0.0,4
202,Bukit Timah,Leedon Park,1.3161,103.7955,6400,0.170931,0.228455,0.431962,0.83,0.0,4
204,Kallang,Crawford,1.3006,103.8641,8780,0.234496,0.160977,0.423078,0.82,0.0,4
205,Pasir Ris,Loyang East,1.3676,103.9726,2160,0.057689,0.296549,0.457875,0.81,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
318,Jurong West,Kian Teck,1.3337,103.6996,0,0.000000,0.000000,0.000000,0.00,0.0,4
319,Toa Payoh,Bidadari,1.3411,103.8756,0,0.000000,0.000000,0.000000,0.00,0.0,4
320,Ang Mo Kio,Yio Chu Kang North,1.3926,103.8472,0,0.000000,0.000000,0.000000,0.00,0.0,4
321,Boon Lay,Liu Fang,1.3188,103.7083,0,0.000000,0.000000,0.000000,0.00,0.0,4
